In [ ]:
#default_exp data.training_data

### Training Data Preparation
> This module reads features computed on customer_id and email, merges both datasets, obtain `is_fraud` label for orders and creates two dataset that are suitable to train the model:

>1) Sampled: a dataset that contains all transactions marked as fraud, plus a subsample of the non fraud orders, in order to obtain a less-imbalanced dataset and train faster

>2) Validation: a dataset containing only most recent orders, containing all fraud and non fraud transactions to be use as a final validation for the trained model.

In [ ]:
#export
from typing import Dict
from datetime import datetime, timezone, timedelta
import random
import math
import dask.dataframe as dd
import numpy as np

from hopeit.app.context import EventContext
from hopeit.app.events import Spawn, SHUFFLE
from hopeit.app.api import event_api
from hopeit.app.logger import app_logger

from fraud_poc.jobs import get_client, FeatureCalcJob, TrainingDataJob

In [ ]:
#export
__steps__ = ['run']

logger = app_logger()

In [ ]:
#export
def _merge_feature_datasets(datasets: Dict[str, str]):
    df = None
    for key, path in datasets.items():
        df_key = dd.read_parquet(path, engine='fastparquet')
        if df is not None:
            df = df.merge(df_key, left_on='order_id', right_on='order_id', suffixes=('', '_DROP'))
            keep_cols = [c for c in df.columns if c[-5:] != '_DROP']
            df = df[keep_cols]
        else:
            df = df_key
    return df


def _add_labels(df):
    df['is_fraud'] = (df['known_ip_addr_by_customer_id'] == 0) & (df['num_ip_addr_by_customer_id'] > 3)
    df['is_fraud'] = df['is_fraud'] | ((df['known_email_by_customer_id'] == 0) & (df['num_email_by_customer_id'] > 3))
    df['is_fraud'] = df['is_fraud'] | (df['order_amount'] > (1. + 0.5 * random.random() * df['order_amount_mean_by_customer_id']))
    df['is_fraud'] = df['is_fraud'].apply(lambda x: int(x & (random.random() > 0.1)), meta=('is_fraud', int))
    return df

def _add_sample_flag(df, subsample_not_fraud: float):
    df['sample'] = df['is_fraud'].apply(lambda x: int((x > 0) | (random.random() > (1.-subsample_not_fraud))), meta=('sample', int))
    return df

def _add_validation_flag(df):
    now = datetime.now(tz=timezone.utc)
    now_epoch = now.timestamp()
    df['now'] = now
    df['elapsed_wgt'] = df['order_date'].apply(lambda x: math.log(max(0.001, 1. - (now_epoch - x.timestamp())/now_epoch)) + 1., meta=('elapsed_wgt', float))
    df['validation'] = df['elapsed_wgt'].apply(lambda x: int((max(0, x)  * random.random()) > 0.8), meta=('validation', int))
    return df

def _add_fold_number(df, num_folds):
    df['fold'] = df['is_fraud'].apply(lambda x: random.randint(0, num_folds), meta=('fold', int)) 
    return df

In [ ]:
#export
def run(job: FeatureCalcJob, context: EventContext) -> TrainingDataJob:
    base_path = context.env['data']['training']
    num_folds = context.env['training_data']['num_folds']
    subsample_not_fraud = context.env['training_data']['subsample_not_fraud']
    
    client = get_client(context)
    try:
        df = _merge_feature_datasets(job.features)
        df = _add_labels(df)
        df = _add_sample_flag(df, subsample_not_fraud)
        df = _add_validation_flag(df)
        df = _add_fold_number(df, num_folds)
        
        sampled_save_path = f"{base_path}/sampled/"
        logger.info(context, f"Saving sampled training dataset to {sampled_save_path}...")
        df_sample = df[df['sample'] > 0]
        df_sample = df_sample.set_index('fold')
        df_sample.to_parquet(sampled_save_path)
        
        valid_save_path = f"{base_path}/validation/"
        logger.info(context, f"Saving weighted validation dataset to {valid_save_path}...")
        df_validation = df[df['validation'] >0 ]
        df_validation.to_parquet(valid_save_path)

        return TrainingDataJob(
            sources=job.features,
            sampled=sampled_save_path,
            validation=valid_save_path
        )
    except Exception as e:
        logger.error(context, e)
        return None
    finally:
        client.close()

### Test from notebook

In [ ]:
from hopeit.testing.apps import config, execute_event

app_config = config('config/training-pipeline.json')
job = FeatureCalcJob(sources={'customer_id': './data/partitioned/customer_id/', 'email': './data/partitioned/email'}, 
                     features={'customer_id': './data/features/customer_id/', 'email': './data/features/email/'})
result = await execute_event(app_config, 'data.training-data', job)
result

2020-07-07 22:36:58,442 | INFO | fraud-poc 0.0.1-training data.training-data leo-legion 18704 | Saving sampled training dataset to ./data/training/sampled/... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-07T22:36:57.211612+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=
2020-07-07 22:37:43,979 | INFO | fraud-poc 0.0.1-training data.training-data leo-legion 18704 | Saving weighted validation dataset to ./data/training/validation/... | track.operation_id=test_operation_id | track.request_id=test_request_id | track.request_ts=2020-07-07T22:36:57.211612+00:00 | stream.name= | stream.msg_id= | stream.consumer_group=


TrainingDataJob(sources={'customer_id': './data/features/customer_id/', 'email': './data/features/email/'}, sampled='./data/training/sampled/', validation='./data/training/validation/')

In [ ]:
dd.read_parquet(result.sampled).describe().compute()

Task was destroyed but it is pending!
task: <Task pending name='Task-625' coro=<HTTP1ServerConnection._server_request_loop() running at /opt/dev/anaconda3/envs/dask/lib/python3.8/site-packages/tornado/http1connection.py:817> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fb0c8c25af0>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /opt/dev/anaconda3/envs/dask/lib/python3.8/site-packages/tornado/ioloop.py:690]>
Task was destroyed but it is pending!
task: <Task pending name='Task-630' coro=<HTTP1ServerConnection._server_request_loop() running at /opt/dev/anaconda3/envs/dask/lib/python3.8/site-packages/tornado/http1connection.py:817> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7fb0a3fa0a60>()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /opt/dev/anaconda3/envs/dask/lib/python3.8/site-packages/tornado/ioloop.py:690]>
Task was destroyed but it is pending!
task: <Task pending name='Task-631' coro=<HTTP1ServerConnection._server_request_loop() running a

,order_amount,num_email_by_customer_id,num_ip_addr_by_customer_id,same_email_by_customer_id,same_ip_addr_by_customer_id,known_email_by_customer_id,known_ip_addr_by_customer_id,order_amount_mean_by_customer_id,order_amount_std_by_customer_id,order_amount_min_by_customer_id,...,known_customer_id_by_email,order_amount_mean_by_email,order_amount_std_by_email,order_amount_min_by_email,order_amount_max_by_email,order_amount_sum_by_email,is_fraud,sample,elapsed_wgt,validation
count,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,...,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1.072089e+06,1072089.0,1.072089e+06,1.072089e+06
mean,5.267080e+02,2.160165e+00,3.119257e+00,5.705758e-01,4.602976e-01,9.707590e-01,8.614052e-01,5.012814e+02,2.656489e+02,1.001119e+02,...,9.895568e-01,5.018918e+02,2.634956e+02,1.052752e+02,8.979406e+02,4.789087e+03,8.804055e-01,1.0,9.901491e-01,1.927834e-01
std,2.759189e+02,1.013540e+00,1.369770e+00,4.949942e-01,4.984215e-01,1.684815e-01,3.455234e-01,9.665022e+01,5.215631e+01,9.980408e+01,...,1.016568e-01,9.983118e+01,5.651710e+01,1.096340e+02,1.074631e+02,1.212657e+03,3.244870e-01,0.0,5.692438e-03,3.944846e-01
min,1.560899e-03,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.001237e+00,0.000000e+00,2.271392e-04,...,0.000000e+00,3.408705e-02,0.000000e+00,2.271392e-04,3.408705e-02,3.408705e-02,0.000000e+00,1.0,9.802699e-01,0.000000e+00
25%,2.959992e+02,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,4.384200e+02,2.375031e+02,3.050219e+01,...,1.000000e+00,4.383146e+02,2.362462e+02,3.125854e+01,8.618774e+02,4.216201e+03,1.000000e+00,1.0,9.852614e-01,0.000000e+00
50%,5.319706e+02,2.000000e+00,3.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,5.021227e+02,2.701232e+02,7.207299e+01,...,1.000000e+00,5.028616e+02,2.695697e+02,7.425120e+01,9.296629e+02,4.922503e+03,1.000000e+00,1.0,9.901895e-01,0.000000e+00
75%,7.665151e+02,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.661961e+02,3.000172e+02,1.407146e+02,...,1.000000e+00,5.675166e+02,2.998108e+02,1.457638e+02,9.703124e+02,5.584035e+03,1.000000e+00,1.0,9.951112e-01,0.000000e+00
max,9.999995e+02,7.000000e+00,1.000000e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,9.999918e+02,4.982714e+02,9.999918e+02,...,1.000000e+00,9.999918e+02,4.982714e+02,9.999918e+02,9.999995e+02,8.915618e+03,1.000000e+00,1.0,9.999744e-01,1.000000e+00


In [ ]:
dd.read_parquet(result.validation).describe().compute()

,order_amount,num_email_by_customer_id,num_ip_addr_by_customer_id,same_email_by_customer_id,same_ip_addr_by_customer_id,known_email_by_customer_id,known_ip_addr_by_customer_id,order_amount_mean_by_customer_id,order_amount_std_by_customer_id,order_amount_min_by_customer_id,...,order_amount_mean_by_email,order_amount_std_by_email,order_amount_min_by_email,order_amount_max_by_email,order_amount_sum_by_email,is_fraud,sample,elapsed_wgt,validation,fold
count,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,...,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.000000,229899.0,229899.000000
mean,499.575716,2.157073,3.111566,0.572934,0.464047,0.972679,0.867242,499.597151,267.026925,97.229817,...,499.416451,264.900245,101.846350,897.765100,4777.521732,0.785497,0.892153,0.990291,1.0,5.000183
std,288.783726,1.012125,1.365872,0.494653,0.498707,0.163017,0.339314,96.555412,51.819866,97.441246,...,99.781962,55.895181,106.629882,107.388765,1202.931560,0.410478,0.310188,0.005683,0.0,3.162667
min,0.004208,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.457344,0.000000,0.000227,...,0.295380,0.000000,0.000227,0.295380,0.295380,0.000000,0.000000,0.980270,1.0,0.000000
25%,254.207715,1.000000,2.000000,0.000000,0.000000,1.000000,1.000000,437.617167,239.067854,29.753422,...,435.394575,238.285830,30.483997,862.305111,4199.806490,1.000000,1.000000,0.985520,1.0,2.000000
50%,505.215247,2.000000,3.000000,1.000000,0.000000,1.000000,1.000000,501.715618,271.648605,70.648778,...,500.704623,271.126161,71.581809,930.681119,4909.946848,1.000000,1.000000,0.990526,1.0,5.000000
75%,755.731631,3.000000,4.000000,1.000000,1.000000,1.000000,1.000000,565.428195,301.652860,137.977865,...,565.241917,301.407646,141.064595,970.713671,5579.990289,1.000000,1.000000,0.995357,1.0,8.000000
max,999.996485,7.000000,10.000000,1.000000,1.000000,1.000000,1.000000,997.396879,498.271440,997.396879,...,997.812768,498.271440,997.812768,999.999489,8880.975814,1.000000,1.000000,0.999974,1.0,10.000000


In [ ]:
df = dd.read_parquet('./data/training/validation/')
df[df.is_fraud == 0].head()

,order_id,order_date,email,ip_addr,order_amount,customer_id,email_by_customer_id,ip_addr_by_customer_id,num_email_by_customer_id,num_ip_addr_by_customer_id,...,order_amount_min_by_email,order_amount_max_by_email,order_amount_sum_by_email,order_amount_by_email,is_fraud,sample,now,elapsed_wgt,validation,fold
55,eef0e5b1-9898-4c9a-8e92-81bac714a870,2020-03-18 06:20:25+00:00,afe4b272a5def9d9b3cd9d8ac9b6136bbe90ffe9,c7ee6b54b39b1b5edefa3aa85d24665ed8feadd5,547.086027,002d6c90-2680-494a-b65e-c969c48277c8,"['afe4b272a5def9d9b3cd9d8ac9b6136bbe90ffe9', '...","['c7ee6b54b39b1b5edefa3aa85d24665ed8feadd5', '...",1,1,...,46.870327,698.002805,4037.924935,"[259.83261171744243, 46.870327361839934, 630.1...",0,0,2020-07-08 10:40:24.712622+00:00,0.993902,1,9
78,442b9d8c-fc9e-40a9-ae6d-442b8f222a73,2020-05-30 14:43:39+00:00,414a4d901fb5d28d0b794c0b145430a1b9eceffa,504395f5e6b0426c6e5917bd18a287abedb7d7f8,489.182172,003454f4-d2cc-4845-8256-15a7f639becc,"['414a4d901fb5d28d0b794c0b145430a1b9eceffa', '...","['504395f5e6b0426c6e5917bd18a287abedb7d7f8', '...",2,2,...,18.227239,811.657061,4416.252078,"[510.21130492364875, 155.86290494965394, 491.4...",0,1,2020-07-08 10:40:24.712622+00:00,0.997893,1,2
148,d01377a8-2b7f-45d9-bedb-d32a39af893e,2019-11-20 06:07:59+00:00,bf8edae2c642ce476fba0404cd48ebd306a9e331,331e631a8879c97c23fad416f8a96a9685f5186d,565.939878,005151c1-316c-4b11-a36e-16c495bc5528,"['7fb135d0fbc29f0fe2b64fcab2ed929928f62aaf', '...","['f5f788c173e5c89122cb3502e2c1205dd00aa52d', '...",2,4,...,35.432090,800.632491,3813.130451,"[102.4452576754118, 305.81444470653696, 468.17...",0,0,2020-07-08 10:40:24.712622+00:00,0.987391,1,1
300,e52cd433-60fe-428c-a027-1b86f084138b,2019-11-20 21:06:08+00:00,031a29720199b6e87af8abe6ac60d9725b920a88,199659be876d5da3b23132215225dcef9b8e64a0,969.332707,00d8e94f-f19f-4d6d-b225-ece8ca2e80a7,"['031a29720199b6e87af8abe6ac60d9725b920a88', '...","['199659be876d5da3b23132215225dcef9b8e64a0', '...",5,6,...,142.240825,969.332707,5138.391860,"[228.43338205886343, 315.28410476759495, 641.1...",0,1,2020-07-08 10:40:24.712622+00:00,0.987425,1,5
354,ccb272c0-4c82-4719-ae5a-bcd6cda09962,2020-05-17 21:21:06+00:00,daf294993afcba23204d924db0139ecb1600f3ac,3750a44f6dcd89f0849caf09d12ad25e0a55305c,284.537458,00f9c62c-572d-46d1-b04f-bea893fbe0eb,"['ac35cd18ba6c709b33cbb1da170a4405fb2836cf', '...","['826a6db07e5f5f62ba0f3263ff5b62a773573d77', '...",5,6,...,116.431429,946.398526,4239.615681,"[208.65553859182018, 298.96564343692825, 650.7...",0,0,2020-07-08 10:40:24.712622+00:00,0.997202,1,7
